# 9. Serverless

## 9.3 TensorFlow Lite

In [1]:
import tensorflow as tf
from tensorflow import keras
tf.__version__

'2.14.0'

In [2]:
!conda list | grep "keras"

keras                     2.14.0             pyhd8ed1ab_0    conda-forge
keras-image-helper        0.0.1                    pypi_0    pypi


In [3]:
!pip list | grep "keras"

keras                   2.14.0
keras-image-helper      0.0.1


Version of TF should be above 2.7.0

In [4]:
!python -V

Python 3.11.10


In [ ]:
# Load the model
# !wget https://www.github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter-7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

In [ ]:
# Load sample image
# !wget http://bit.ly/mlbookcamp-pants -O ./images/pants.jpg

--2024-09-23 02:05:07--  http://bit.ly/mlbookcamp-pants
Auflösen des Hostnamens bit.ly (bit.ly)… 67.199.248.10, 67.199.248.11
Verbindungsaufbau zu bit.ly (bit.ly)|67.199.248.10|:80 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 301 Moved Permanently
Platz: https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg [folgend]
--2024-09-23 02:05:08--  https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg
Auflösen des Hostnamens raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Verbindungsaufbau zu raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 23048 (23K) [image/jpeg]
Wird in »./images/pants.jpg« gespeichert.

./images/pants.jpg  100%[====

In [5]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.applications.xception import preprocess_input

In [6]:
model = keras.models.load_model('./models/xception_v4_08_0.868.h5')

2024-12-04 07:46:03.077403: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-04 07:46:03.077477: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-04 07:46:03.077490: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-04 07:46:03.077547: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-04 07:46:03.077568: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
img = load_img('./images/pants.jpg', target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [8]:
preds = model.predict(X)

2024-12-04 07:46:11.060567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step


In [9]:
preds

array([[-2.1288574 , -4.6540356 , -1.1132694 , -0.20497966,  9.407786  ,
        -0.106648  , -5.7733626 ,  2.9702551 , -1.4123498 , -4.4523616 ]],
      dtype=float32)

In [10]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [11]:
dict(zip(classes, preds[0]))

{'dress': -2.1288574,
 'hat': -4.6540356,
 'longsleeve': -1.1132694,
 'outwear': -0.20497966,
 'pants': 9.407786,
 'shirt': -0.106648,
 'shoes': -5.7733626,
 'shorts': 2.9702551,
 'skirt': -1.4123498,
 't-shirt': -4.4523616}

Nothing new so far, just testing if it works.

## Convert Keras to TF-Lite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('./models/clothing_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/r8/zdlnr35s6qz6zx67nmc9bdnm0000gn/T/tmpgx4moqy0/assets


INFO:tensorflow:Assets written to: /var/folders/r8/zdlnr35s6qz6zx67nmc9bdnm0000gn/T/tmpgx4moqy0/assets
2024-12-04 07:46:36.854461: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-12-04 07:46:36.854649: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-12-04 07:46:36.855211: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/r8/zdlnr35s6qz6zx67nmc9bdnm0000gn/T/tmpgx4moqy0
2024-12-04 07:46:36.867718: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-12-04 07:46:36.867736: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/r8/zdlnr35s6qz6zx67nmc9bdnm0000gn/T/tmpgx4moqy0
2024-12-04 07:46:36.892406: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2024-12-04 07:46:36.904190: I tensorflow/cc/saved_model/load

In [13]:
import tensorflow.lite as tflite

allocate the memory for the model

In [14]:
# Load the TFLite model and get indices of input and output tensors
interpreter = tflite.Interpreter("./models/clothing_model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
VERBOSE: Replacing 104 out of 104 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions for the whole graph.
INFO: Successfully applied the default TensorFlow Lite delegate indexed at 0.
 *NOTE*: because a delegate has been applied, the precision of computations should be unchanged, but the exact output tensor values may have changed. If such output values are checked in your code, like in your tests etc., please consider increasing error tolerance for the check.


invoke the converter and fetch the results

In [15]:
# Inferencing
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [16]:
# Post-processing: get the class name of the highest probability

classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -2.2917588,
 'hat': -2.2173786,
 'longsleeve': -1.9650993,
 'outwear': -1.4446676,
 'pants': 7.1728725,
 'shirt': -0.056686,
 'shoes': -4.422337,
 'shorts': 2.8628511,
 'skirt': -2.050411,
 't-shirt': -3.4516563}

Now we used TensorFlow Lite to make the predictions.

## Removing TF dependency

In [17]:
from PIL import Image

In [18]:
with Image.open('./images/pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

In [19]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [20]:
x = np.array(img, dtype="float32")
X = np.array([x])

X = preprocess_input(X)

In [21]:
# Inferencing
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [22]:
# Post-processing: get the class name of the highest probability
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -2.2917588,
 'hat': -2.2173786,
 'longsleeve': -1.9650993,
 'outwear': -1.4446676,
 'pants': 7.1728725,
 'shirt': -0.056686,
 'shoes': -4.422337,
 'shorts': 2.8628511,
 'skirt': -2.050411,
 't-shirt': -3.4516563}

## Simpler way of doing it

- go to Alexey's github profile
- open repositories
- look for ["keras-image-helper"](https://github.com/alexeygrigorev/keras-image-helper)

In [46]:
# !pip install keras-image-helper

In [25]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
ERROR: Could not find a version that satisfies the requirement tflite_runtime (from versions: none)
ERROR: No matching distribution found for tflite_runtime


To not use the TF Lite module from TensorFlow, we install the tflite_runtime package.

In [24]:
!conda list | grep "tflite"

In [23]:
# import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

ModuleNotFoundError: No module named 'tflite_runtime'

In [ ]:
# Load the TFLite model and get indices of input and output tensors
interpreter = tflite.Interpreter("./models/clothing_model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [ ]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
# Post-processing: get the class name of the highest probability
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

Next we convert this notebook to a Python script.